# Classifieur par SIFT (Taches)

## Dépendances

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
import time

df = pd.read_csv('base_de_données.csv')
images = df['photo']

df_train = pd.read_csv('X_train.csv')
df_test = pd.read_csv('X_test.csv')
indices_train = df_train['index']
indices_test = df_test['index']

images_train = df_train['photo']
images_test = df_test['photo']


#sift = cv2.SIFT_create(edgeThreshold=2.4,contrastThreshold=0.011)
sift = cv2.SIFT_create(edgeThreshold=2,contrastThreshold=0.014,sigma=0.5)

debut = time.time()

## Création des descripteurs

In [3]:
buff =  cv2.imread('./Photographies pieces chaudes/' + images[0] ,cv2.IMREAD_GRAYSCALE)

mask = np.zeros_like(buff)
z = np.ones((550,600))
mask[350:900,600:1200] = z
mask = np.uint8(mask)

i = 0
for index,image in zip(indices_train,images_train):
	if i==0:
		img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
		#kp,des = sift.detectAndCompute(img,None)
		kp = sift.detect(img,mask = mask)
		kp,des = sift.compute(img,kp)
		descripteurs = des		
	else:
		img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
		#kp,des = sift.detectAndCompute(img,None)
		kp = sift.detect(img,mask = mask)
		kp,des = sift.compute(img,kp)		
		descripteurs = np.concatenate((descripteurs,des),axis=0)
	i+=1
	print(i)

descripteurs = np.float32(descripteurs)
print(descripteurs.shape) #(22705,128)
np.save('descripteurs.npy',descripteurs)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
(18736, 128)


## Création des labels

In [4]:
#Creation des labels (tache : non conforme (1) si q_tache>3)
q_tache_train = df_train['q_tache'].fillna(0)
Y_train = q_tache_train.to_numpy(np.float32)
Y_train = (Y_train >3)
Y_train = np.int16(Y_train)

q_tache_test = df_test['q_tache'].fillna(0)
Y_test = q_tache_test.to_numpy(np.float32)
Y_test = (Y_test >3)
Y_test = np.int16(Y_test)


np.save('Y_train.npy',Y_train)
np.save('Y_test.npy',Y_test)

## Création des histogrammes sur les jeux

In [5]:
descripteurs = np.load('descripteurs.npy')

#Calculs des center des k means
nbre_clusters = 500

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center=cv2.kmeans(descripteurs,nbre_clusters,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
np.save('center_k_means.npy',center)

Y_train = np.load('Y_train.npy')
n = Y_train.shape[0]
X_train = np.zeros((n,nbre_clusters))

center_k_means = np.load('center_k_means.npy')

k=0
for indice,image in zip(indices_train,images_train):	
	img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
	#kp,des = sift.detectAndCompute(img,None)
	kp = sift.detect(img,mask = mask)
	kp,des = sift.compute(img,kp)
	nbr_des = des.shape[0]
	x = np.zeros((nbre_clusters,))
	for elem in des:
		distance = np.linalg.norm(center_k_means - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	X_train[k] = x
	k+=1
	print(k)

np.save('X_train.npy',X_train)
print(X_train.shape)
fin_train = time.time()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
(133, 500)


In [6]:
m = len(images)-X_train.shape[0]
X_test = np.zeros((m,nbre_clusters))


k=0
for indice,image in zip(indices_test,images_test):	

	img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
	#kp,des = sift.detectAndCompute(img,None)
	kp = sift.detect(img,mask = mask)
	kp,des = sift.compute(img,kp)
	nbr_des = des.shape[0]
	x = np.zeros((nbre_clusters,))
	for elem in des:
		distance = np.linalg.norm(center_k_means - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	X_test[k] = x
	k+=1
	print(k)
	
np.save('X_test.npy',X_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


## Classifieurs
### LDA

In [16]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis()

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

LDA.fit(X_train,Y_train)
Y_pred = LDA.predict(X_test)


def matrice_confusion(pred,lab):
	#calcul des vrais positifs
	vp = np.count_nonzero((pred == 1) & (lab == 1))
	#calculs des faux positifs
	fp = np.count_nonzero((pred == 1) & (lab == 0))
	#calculs des vrais négatifs
	vn = np.count_nonzero((pred == 0) & (lab == 0))
	#calculs des faux négatifs
	fn = np.count_nonzero((pred == 0) & (lab == 1))
	return [vp,fp,vn,fn]
	
conf = matrice_confusion(Y_pred,Y_test)
print(conf)
taux_fn = conf[3]/sum(conf)
taux_fn_neg = conf[3]/(conf[3]+conf[0])
non_conf = conf[0]+conf[3]
conf = conf[1]+conf[2]

score_random = max(conf/(conf+non_conf),non_conf/(conf+non_conf))

print(f"Non conformes : {non_conf}")
print(f"Conformes : {conf}")
print(f"taux de faux conformes par rapport aux non conformes : {taux_fn_neg}")

print(f"score = {LDA.score(X_test,Y_test)}")
print(f"score du random = {score_random}")


[27, 4, 6, 9]
Non conformes : 36
Conformes : 10
taux de faux conformes par rapport aux non conformes : 0.25
score = 0.717391304347826
score du random = 0.782608695652174


Précisions pour SIFT défaut:
precision 500 clusters : 0.60  
precision 50 clusters : 0.65  
precision 35 clusters : 0.56  
precision 100 clusters : 0.64  
precision 75 clusters : 0.57

Précision pour SIFT modifié Taches:  
75 clusters :0.54  
50 clusters :0.53846  
500 clusters : 0.68531

Précision pour SIFT modifié 2 (sigma = 0.5):  
500 clusters : 0.68   
50 clusters : 0.59

1-NN

In [18]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

neigh.fit(X_train,Y_train)
Y_pred = neigh.predict(X_test)

conf = matrice_confusion(Y_pred,Y_test)
print(conf)
taux_fn = conf[3]/sum(conf)
taux_fn_neg = conf[3]/(conf[3]+conf[0])
non_conf = conf[0]+conf[3]
conf = conf[1]+conf[2]

score_random = max(conf/(conf+non_conf),non_conf/(conf+non_conf))

print(f"Non conformes : {non_conf}")
print(f"Conformes : {conf}")
print(f"taux de faux conformes par rapport aux non conformes : {taux_fn_neg}")

print(f"score = {neigh.score(X_test,Y_test)}")
print(f"score du random = {score_random}")

[20, 3, 7, 16]
Non conformes : 36
Conformes : 10
taux de faux conformes par rapport aux non conformes : 0.4444444444444444
score = 0.5869565217391305
score du random = 0.782608695652174


score   
1-NN : 0.706  
2-NN : 0.594    
3-NN : 0.741   
4-NN : 0.622  
5-NN : 0.741    
10-NN : 0.678
